In [1]:
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

from dna_data import DeepSEA_Dataset
from model import *

In [2]:
import sys
import logging
import datetime

log = logging.getLogger('snowdeer_log')
log.setLevel(logging.DEBUG)

log_file_name=sys.argv[-1].split('/')[-1].split('.')[-2].split('-')[-1]+'_'+datetime.datetime.now().strftime('%Y%m%d_%H%M%S')+'.log'
fileHandler = logging.FileHandler(log_file_name)
streamHandler = logging.StreamHandler()

formatter = logging.Formatter(' %(asctime)s [%(levelname)s] (%(filename)s:%(lineno)d) > %(message)s')
fileHandler.setFormatter(formatter)
streamHandler.setFormatter(formatter)

log.addHandler(fileHandler)
log.addHandler(streamHandler)

#log.debug('debug')

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
model=RNN()
model.to(device)

RNN(
  (lstm): LSTM(4, 32, num_layers=2, dropout=0.8, bidirectional=True)
  (dropout): Dropout(p=0.8, inplace=False)
  (fc): Linear(in_features=128, out_features=919, bias=True)
)

In [5]:
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,weight_decay=5e-07)
optimizer = optim.Adagrad(model.parameters(),weight_decay=5e-07)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5,verbose=1)
loss_function = nn.BCEWithLogitsLoss()

In [6]:
BATCH_SIZE=100
NUM_WORKERS=4

dataloader_valid = DataLoader(
    dataset=DeepSEA_Dataset('data/valid.mat','validxdata','validdata'), 
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)

In [ ]:
TOTAL_EPOCH=10

for epoch in range(TOTAL_EPOCH):
    train_loss_list=[]
    valid_loss_list=[]
    for i in range(10):
        # train data load
        try:
            del dataloader_train,dataset_train
        except:
            pass
        dataset_train=DeepSEA_Dataset('data/train_{}.pt'.format(i),'trainxdata','traindata')
        dataloader_train=DataLoader(dataset=dataset_train,shuffle=True,batch_size=BATCH_SIZE,num_workers=NUM_WORKERS)

        for batch_idx,(batch_X,batch_Y) in enumerate(dataloader_train):
            if model.batch_first==False:
                batch_X=batch_X.transpose(1,2).transpose(0,1)
            batch_X=batch_X.to(device) 
            batch_Y=batch_Y.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            output=model(batch_X)
            loss = loss_function(output, batch_Y)
            loss.backward()
            optimizer.step()  
            
            train_loss_list.append(loss.item())
            
            if batch_idx%100==99:
                log.info('epoch: {:d}, dataset_split: {:d}, batch: {:d}, progress: {:.2f}% loss: {:.3f}'.format(epoch,i,batch_idx,100*(i*len(dataloader_train)+batch_idx)/(10*len(dataloader_train)),np.mean(train_loss_list)))
                train_loss_list.clear()
        
        with torch.no_grad():
            for batch_idx, (batch_X, batch_Y) in enumerate(dataloader_valid):
                if model.batch_first==False:
                    batch_X = batch_X.transpose(1,2).transpose(0,1).to(device)
                else:
                    batch_X = batch_X.to(device)
                batch_Y = batch_Y.to(device)

                output = model(batch_X)
                loss = loss_function(output, batch_Y)
                valid_loss_list.append(loss.item())
                
            print(log.info('epoch: {:d}, dataset_split: {:d}, valid loss: {:.3f}'.format(epoch,i,np.mean(valid_loss_list))))
            valid_loss_list.clear()

 2019-12-30 02:48:39,377 [INFO] (<ipython-input-7-c013bdf76e82>:33) > epoch: 0, dataset_split: 0, batch: 99, progress: 0.23% loss: 0.131
 2019-12-30 02:48:56,050 [INFO] (<ipython-input-7-c013bdf76e82>:33) > epoch: 0, dataset_split: 0, batch: 199, progress: 0.45% loss: 0.102
 2019-12-30 02:49:12,572 [INFO] (<ipython-input-7-c013bdf76e82>:33) > epoch: 0, dataset_split: 0, batch: 299, progress: 0.68% loss: 0.101
 2019-12-30 02:49:29,057 [INFO] (<ipython-input-7-c013bdf76e82>:33) > epoch: 0, dataset_split: 0, batch: 399, progress: 0.91% loss: 0.104
 2019-12-30 02:49:45,548 [INFO] (<ipython-input-7-c013bdf76e82>:33) > epoch: 0, dataset_split: 0, batch: 499, progress: 1.13% loss: 0.101
 2019-12-30 02:50:02,255 [INFO] (<ipython-input-7-c013bdf76e82>:33) > epoch: 0, dataset_split: 0, batch: 599, progress: 1.36% loss: 0.099
 2019-12-30 02:50:19,036 [INFO] (<ipython-input-7-c013bdf76e82>:33) > epoch: 0, dataset_split: 0, batch: 699, progress: 1.59% loss: 0.101
 2019-12-30 02:50:35,621 [INFO] (<i